In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/imdb-reviews-pt-br.zip")

In [3]:
df.shape

(49459, 4)

In [4]:
df.sentiment.unique()

array(['neg', 'pos'], dtype=object)

In [5]:
df['label'] = df.sentiment.apply(lambda x: 0 if x =='neg' else 1)

In [6]:
df.tail(5)

,id,text_en,text_pt,sentiment,label
49454,49456,"Seeing as the vote average was pretty low, and...","Como a média de votos era muito baixa, e o fat...",pos,1
49455,49457,"The plot had some wretched, unbelievable twist...",O enredo teve algumas reviravoltas infelizes e...,pos,1
49456,49458,I am amazed at how this movieand most others h...,Estou espantado com a forma como este filme e ...,pos,1
49457,49459,A Christmas Together actually came before my t...,A Christmas Together realmente veio antes do m...,pos,1
49458,49460,Working-class romantic drama from director Mar...,O drama romântico da classe trabalhadora do di...,pos,1


# Testando o modelo com acuracia

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split


# Dados de treinamento (texto e suas respectivas labels)
text_data = df.text_pt.values
labels = df.label.values

# Divide os dados em treinamento e teste
text_train, text_test, labels_train, labels_test = train_test_split(text_data, labels, test_size=0.2)

# Cria o objeto de vetorização de texto
vectorizer = CountVectorizer()

# Transforma o texto em um vetor de recursos
text_train_vectors = vectorizer.fit_transform(text_train)
text_test_vectors = vectorizer.transform(text_test)

# Cria o classificador
clf = MultinomialNB()

# Treina o classificador
clf.fit(text_train_vectors, labels_train)

# Faz a previsão com os dados de teste
prediction = clf.predict(text_test_vectors)

# Calcula a precisão
accuracy = sum(prediction == labels_test) / len(labels_test)

print("A precisão é:", accuracy)

A precisão é: 0.843509906995552


# Matriz de confusão

In [8]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
# Faz as previsões com os dados de teste
predictions = clf.predict(text_test_vectors)

# Gera a matriz de confusão
cm = confusion_matrix(labels_test, predictions)



In [9]:
cm

array([[4354,  633],
       [ 915, 3990]], dtype=int64)

In [10]:
from sklearn.metrics import accuracy_score
accuracy_score(labels_test, predictions)

0.843509906995552

# GridSearchCV

In [16]:
from sklearn.model_selection import GridSearchCV

# Dados de treinamento (texto e suas respectivas labels)
text_data = df.text_pt.values
labels = df.label.values

# Cria o objeto de vetorização de texto
vectorizer = CountVectorizer()

# Transforma o texto em um vetor de recursos
text_vectors = vectorizer.fit_transform(text_data)

# Define a lista de parâmetros a serem testados
parameters = {'alpha': [0.1, 0.5, 1.0, 2.0]}

# Cria o classificador
clf = MultinomialNB()

# Instância o GridSearchCV
grid_search = GridSearchCV(clf, parameters, cv=5)

# Treina o modelo com o GridSearchCV
grid_search.fit(text_vectors, labels)

# Imprime os melhores parâmetros encontrados pelo GridSearchCV
print("Melhores parâmetros encontrados: ", grid_search.best_params_)

# Faz as previsões com o melhor modelo encontrado pelo GridSearchCV
predictions = grid_search.predict(text_vectors)

# Imprime a acurácia do modelo
print("Acurácia: ", np.mean(predictions == labels))

Melhores parâmetros encontrados:  {'alpha': 2.0}
Acurácia:  0.8902525324005742


# validação cruzada

In [11]:
from sklearn.model_selection import cross_val_score

# Dados de treinamento (texto e suas respectivas labels)
text_data = df.text_pt.values
labels = df.label.values

# Cria o objeto de vetorização de texto
vectorizer = CountVectorizer()

# Transforma o texto em um vetor de recursos
text_vectors = vectorizer.fit_transform(text_data)

# Cria o classificador
clf = MultinomialNB()

# Faz a validação cruzada
scores = cross_val_score(clf, text_vectors, labels, cv=5)

print("Média da precisão:", scores.mean())
print("Desvio padrão da precisão:", scores.std())

'''
Neste exemplo, a validação cruzada é feita usando a função cross_val_score do scikit-learn.
 A precisão é avaliada com 5 conjuntos de treinamento e teste (isto pode ser ajustado alterando o parâmetro cv). 
 O resultado é a média da precisão e o desvio padrão da precisão.
'''

Média da precisão: 0.8025635410609695
Desvio padrão da precisão: 0.007609498524722115


'\nNeste exemplo, a validação cruzada é feita usando a função cross_val_score do scikit-learn.\n A precisão é avaliada com 5 conjuntos de treinamento e teste (isto pode ser ajustado alterando o parâmetro cv). \n O resultado é a média da precisão e o desvio padrão da precisão.\n'

In [19]:
grid_search.best_params_['alpha']

2.0

In [20]:
# Dados de treinamento (texto e suas respectivas labels)
text_data = df.text_pt.values
labels = df.label.values

# Cria o objeto de vetorização de texto
vectorizer = CountVectorizer()

# Transforma o texto em um vetor de recursos
text_vectors = vectorizer.fit_transform(text_data)

# Cria o classificador
# Melhores parâmetros encontrados:  {'alpha': 2.0}
clf = MultinomialNB(alpha=grid_search.best_params_['alpha'])

# Treina o classificador
clf.fit(text_vectors, labels)

MultinomialNB(alpha=2.0)

In [21]:
# Dados de teste (texto)
new_text = ["Hoje é um bom dia"]

# Transforma o novo texto em um vetor de recursos
new_text_vector = vectorizer.transform(new_text)

# Faz a previsão
prediction = clf.predict(new_text_vector)

print("A previsão é:", prediction[0])

A previsão é: 1


In [22]:
# Dados de teste (texto)
new_text = ["Hoje o dia está muito ruim"]

# Transforma o novo texto em um vetor de recursos
new_text_vector = vectorizer.transform(new_text)

# Faz a previsão
prediction = clf.predict(new_text_vector)

print("A previsão é:", prediction[0])

A previsão é: 0


In [23]:
import joblib
# Salva o modelo e o vetorizador
joblib.dump(clf, 'model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')

['vectorizer.pkl']